In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import numba 

## Part 1

first draft on assignment 1a, wrote it as an iterative scheme over time and space, saving the two previous time values for the computation of the next time point

In [8]:
def spat_approx_1a(deltax, time, solutions, x, L):
    # if np.isclose(x, deltax): 
    #     # border condition 0
    #     return (func(2*deltax, time) - 2 * solutions[1] + func(0, time))/np.power(deltax, 2)
    # elif np.isclose(x, L-deltax): 
    #     # border condition 1
    #     return ( func(L- 2*deltax, time)-2*func(L-deltax, time) + func(L, time))/np.power(deltax, 2)
    # else:
        # middle point
    return (solutions[2] - 2*solutions[1] + solutions[0])/np.power(deltax, 2)


In [9]:
def time_approx_1a(deltat, time, func, x):
    return (func(x, time+deltat) - 2*func(x, time) + func(x, time-deltat))/np.power(deltat, 2)

thus, the discretized wave equation becomes: 

time_approx_1a(deltat, time, func, x) = c^2* spat_approx_1a(deltax, time, func, x, L)

you can rewrite this to get the values of the next time point

In [ ]:
L = 1
N = 100
c=2
time = 0

deltax = L/N
deltat = L/N
xs = np.arange(0, L, deltax)

#saving the solutions
sols_prev = np.zeros(len(xs))
sols = np.zeros(len(xs))
sols_next = np.zeros(len(xs))

#the full function 
for i in range(1000):
    for j,x in enumerate(xs):
        if j == 0: 
            sols_next[j]= 0
        elif j == len(xs) -1:
            sols_next[j] = 0
        else:
            sols_next[j] = np.power(deltat, 2) * np.power(c, 2) * spat_approx_1a(deltax, time, (sols[j-1], sols[j], sols[j+1]), x, L) + 2*sols[j] - sols_prev[j]

    sols_prev = sols_next.copy()
    sols = sols_next.copy()
    

### B

Euler's method 

f_x+1 = f_x + h * f'_x

(c*delta t)/delta x < 1 for stable computation 
looking at this method, you can derive that in the first two time-points, the function follows the exact same curve as the derivative (f'_x) is 0

In [16]:

def b_one(x):
    return np.sin(2*np.pi*x)

def b_two(x):
    return np.sin(5*np.pi*x)

L = 1
N = 100
c=1
time = 0

deltax = L/N
deltat = 0.001
xs = np.arange(0, L, deltax)

#saving the solutions
sols_prev = [b_one(xje) for xje in xs]
sols = sols_prev.copy()

sols_next = np.zeros(len(xs))

overall_solutions = []

#the full function 
for i in range(10000):
    for j,x in enumerate(xs):
        if j == 0: 
            sols_next[j]= 0
        elif j == len(xs) -1:
            sols_next[j] = 0
        else:
            sols_next[j] = np.power(deltat, 2) * np.power(c, 2) * spat_approx_1a(deltax, time, (sols[j-1], sols[j], sols[j+1]), x, L) + 2*sols[j] - sols_prev[j]
    if i%1000 == 0: 
        overall_solutions.append(sols)
    sols_prev = sols_next.copy()
    sols = sols_next.copy()

In [ ]:
plt.figure(figsize=(5,5))
for k in range(len(overall_solutions)):
    plt.plot(xs, overall_solutions[k])

plt.title("Numerical Time-stepping Approximation")
plt.xlabel("L")
plt.ylabel(r'$\Psi^n$')
plt.show()